In [56]:
import bagpy
from bagpy import bagreader
from datetime import datetime
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt
import numpy as np
import re

In [33]:
PID = '1_EXP05_Full/'
rosbag = 'rosbag_common_2017-07-14-12-53-49_0.bag'
b = bagreader(PID + rosbag)

[INFO]  Data folder 1_EXP05_Full/rosbag_common_2017-07-14-12-53-49_0 already exists. Not creating.


In [34]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def get_timestamp(file):
    start_year = find_nth(file, '_', 2) + 1
    start_month = find_nth(file, '-', 1) + 1
    start_day = find_nth(file, '-', 2) + 1
    start_hour = find_nth(file, '-', 3) + 1
    start_minute = find_nth(file, '-', 4) + 1
    start_second = find_nth(file, '-', 5) + 1
    
    end_year = find_nth(file, '-', 1)
    end_month = find_nth(file, '-', 2)
    end_day = find_nth(file, '-', 3)
    end_hour = find_nth(file, '-', 4)
    end_minute = find_nth(file, '-', 5)
    end_second = find_nth(file, '_', 3)
    
    year = file[start_year:end_year]
    month = file[start_month:end_month]
    day = file[start_day:end_day]
    hour = file[start_hour:end_hour]
    minute = file[start_minute:end_minute]
    second = file[start_second:end_second]
    
    return year, month, day, hour, minute, second

# y, mo, d, h, mi, s = get_timestamp(rosbag)

In [189]:
def unix_to_datetime(ts):
    return datetime.utcfromtimestamp(ts)

In [37]:
b.topic_table

,Topics,Types,Message Count,Frequency
0,/rosout,rosgraph_msgs/Log,764,3566.585034
1,/sar/game_command,sar_game_command_msgs/GameCommand,3,0.006492
2,/sar/game_state,sar_game_command_msgs/GameState,9,0.387295
3,/sar/jibo/animation,sar_jibo_command_msgs/JiboAnimation,59,0.221483
4,/sar/jibo/health,sar_jibo_command_msgs/JiboHealth,9,0.016653
5,/sar/jibo/lookat,sar_jibo_command_msgs/JiboLookat,5,0.048415
6,/sar/jibo/robot_attention_target,std_msgs/String,5581,10.000272
7,/sar/jibo/speech,sar_jibo_command_msgs/JiboSpeech,71,0.227206
8,/sar/jibo/state,sar_robot_command_msgs/RobotState,10850,19.933294
9,/sar/jibo/verbose_state,std_msgs/String,902,1.803065


In [38]:
csvfiles = []
for t in b.topics:
    data = b.message_by_topic(t)
    csvfiles.append(data)

In [176]:
data_attention = b.message_by_topic('/sar/perception/attention_targets')
data_robotgaze = b.message_by_topic('/sar/jibo/robot_attention_target')

In [203]:
df_attention = pd.read_csv(data_attention)
df_robotgaze = pd.read_csv(data_robotgaze)

In [178]:
def get_region(role, attention_string):
    attention = re.sub(r'\W+', '', attention_string)
    start_region = find_nth(attention, role, 1) + len(role) + len('region')
    end_region = find_nth(attention[start_region:], 'role', 1) + start_region
    end_region = len(attention) if (end_region == -1 or end_region < start_region) else end_region
    region = attention[start_region:end_region]
    return region

In [206]:
def get_human_attention(df, role):

    prev_object = get_region(role, df['attention'].at[0])
    prev_timestamp = float(df['Time'].at[0])
    duration = 0
    start_index = 0

    role_attention = []
    role_timestamp = []
    role_duration = []

    for index, row in df.iterrows():
        curr_object = get_region(role, row['attention'])

        # same attentional object
        if curr_object == prev_object:
            duration = duration + (float(row['Time']) - prev_timestamp)
            prev_timestamp = float(row['Time'])

        # detected change in attentional object
        else:
            # record the object and duration
            role_attention.append(prev_object)
            role_duration.append(duration)
            role_timestamp.append(df['Time'].at[start_index])

            # reset pointers
            prev_object = curr_object
            prev_timestamp = float(row['Time'])

            # reset duration for new object
            duration = 0
            start_index = index

            # print(role_attention[-1], role_duration[-1], role_timestamp[-1])
    
    return role_attention, role_timestamp, role_duration

In [208]:
def get_robot_attention(df):

    prev_object = df['data'].at[0]
    prev_timestamp = float(df['Time'].at[0])
    duration = 0
    start_index = 0

    role_attention = []
    role_timestamp = []
    role_duration = []

    for index, row in df.iterrows():
        curr_object = row['data']

        # same attentional object
        if curr_object == prev_object:
            duration = duration + (float(row['Time']) - prev_timestamp)
            prev_timestamp = float(row['Time'])

        # detected change in attentional object
        else:
            # record the object and duration
            role_attention.append(prev_object)
            role_duration.append(duration)
            role_timestamp.append(df['Time'].at[start_index])

            # reset pointers
            prev_object = curr_object
            prev_timestamp = float(row['Time'])

            # reset duration for new object
            duration = 0
            start_index = index

            # print(role_attention[-1], role_duration[-1], role_timestamp[-1])
    
    return role_attention, role_timestamp, role_duration

In [211]:
c_objects, c_timestamps, c_durations = get_human_attention(df_attention, 'child')
p_objects, p_timestamps, p_durations = get_human_attention(df_attention, 'parent')
r_objects, r_timestamps, r_durations = get_robot_attention(df_robotgaze)

In [216]:
child = pd.DataFrame({'object': c_objects, 'duration': c_durations, 'start': c_timestamps})
parent = pd.DataFrame({'object': p_objects, 'duration': p_durations, 'start': p_timestamps})
robot = pd.DataFrame({'object': r_objects, 'duration': r_durations, 'start': r_timestamps})

In [214]:
child.to_csv(PID + 'child.csv')
robot.to_csv(PID + 'robot.csv')

In [235]:
def get_overlap(df1, df2):
    
    # determine the object with the more frequent changes in gaze
    primary = None
    secondary = None
    if len(df1) > len(df2):
        primary = df1
        secondary = df2
    else:
        primary = df2
        secondary = df1
        
    joint_object = []
    joint_duration = []
    joint_timestamp = []
    
    for index_p, row_p in primary.iterrows():
        for index_s, row_s in secondary.iterrows():
            if (row_s['object'] == row_p['object']) and (float(row_s['start']) <= float(row_p['start']) <= (float(row_s['start']) + float(row_s['duration']))):
                start = max([float(row_p['start']), float(row_s['start'])])
                duration = min([float(row_p['duration']), float(row_s['duration'])])
                
                joint_object.append(row_s['object'])
                joint_timestamp.append(start)
                joint_duration.append(duration)
                
                # print(joint_object[-1], joint_duration[-1], joint_timestamp[-1])
                break

    return joint_object, joint_timestamp, joint_duration    

In [240]:
cr_object, cr_timestamp, cr_durations = get_overlap(child, robot)
cp_object, cp_timestamp, cp_durations = get_overlap(child, parent)
pr_object, pr_timestamp, pr_durations = get_overlap(parent, robot)

screen 0.40010643005371094 1500051312.4604757
screen 0.39984893798828125 1500051317.4605925
screen 0.2001054286956787 1500051324.660481
screen 0.10012435913085938 1500051381.760648
screen 0.7000617980957031 1500051680.8617866
screen 2.199965715408325 1500051681.8618774
screen 1.9001095294952393 1500051687.861935
screen 0.20001506805419922 1500051706.16204
nodetection 7.900673151016235 1500051234.860125
nodetection 8.899699926376343 1500051244.0609415
nodetection 0.19982504844665527 1500051253.860417
robot 0.3002438545227051 1500051256.5608096
screen 0.2001020908355713 1500051263.760254
outside 0.20061492919921875 1500051266.3605356
robot 0.699162483215332 1500051267.9612546
nodetection 0.6999564170837402 1500051284.1603801
nodetection 1.2999536991119385 1500051285.1603632
nodetection 4.5000574588775635 1500051289.1602638
nodetection 0.0 1500051300.6603036
nodetection 0.10003232955932617 1500051304.660397
screen 0.09991288185119629 1500051317.4605925
nodetection 0.29999256134033203 1500

child 0.499969482421875 1500051653.5619714
child 0.09949254989624023 1500051660.162436
child 0.3999142646789551 1500051661.1619544
child 0.6998281478881836 1500051666.062081
screen 1.0000526905059814 1500051678.7617738
screen 0.10001230239868164 1500051684.4618378
screen 0.3999910354614258 1500051687.0619295
screen 1.4000566005706787 1500051687.861935
screen 2.3000192642211914 1500051700.86208
screen 1.0001311302185059 1500051705.062124
screen 0.4002339839935303 1500051706.6619642
child 0.39995360374450684 1500051711.4622624
child 0.300123929977417 1500051712.7620294
child 0.20001530647277832 1500051715.9620938
screen 0.19998979568481445 1500051717.8621056
screen 1.9999163150787354 1500051718.5620663
